In [1]:
from collections import Counter
import numpy as np
import tensorflow.contrib.keras as kr
import tensorflow as tf
import time
from datetime import timedelta
import os
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
savePath = "s3://corpus-2/model"

In [23]:
def loadGloVe(filename, emb_size=50):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = word[0].split(":")[0]
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def train_10_fold(train_data, split_type):
    
    tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, cat_to_id, num_classes, seq_length)
    print(len(tx),len(tx[0]),len(tx[1]))
    
    fold_id = 0
    train_acc = []
    test_acc = []
    
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        train_acc.append(model_train(tx[train_i], ty[train_i],split_type,fold_id))
        test_acc.append(model_evaluate(tx[test_i], ty[test_i],split_type,fold_id,categories))
        
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [4]:
class ZhouCLSTMModel:
    '''
    Implementation proposal of: https://arxiv.org/pdf/1511.08630
    '''
    def __init__(self, embedding,
        conv_size    = 3,
        conv_filters = 300,
        drop_rate    = 0.5,
        lstm_units   = 150):
        '''Constructor.
        # Parameters:
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        conv_filters: Number of convolution filters.
        drop_rate: Drop rate for the final output of the LSTM layer.
        lstm_units: Size of the states of the LSTM layer.
        '''
        self._embedding    = embedding
        self._conv_size    = conv_size
        self._conv_filters = conv_filters
        self._drop_rate    = drop_rate
        self._lstm_units   = lstm_units

    def __call__(self, x_input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, x_input)
        
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size,
            self._conv_filters,
            self._drop_rate,
            self._embedding_tf)
        
        self._lstm_tf = self._create_lstm_layer(
            self._lstm_units,
            self._convolution_tf)
        
        return self._lstm_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('lstm:', str(self._lstm_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self,
        conv_size, num_filters, drop_rate, embedding):
        
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, 1, num_filters]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[num_filters]))

        embedding_expanded = tf.expand_dims(embedding, -1)
        conv = tf.nn.conv2d(
            input=embedding_expanded,
            filter=W,
            strides=[1,1,1,1],
            padding='VALID')
        conv_reduced = tf.reshape(
            tensor=conv,
            shape=[-1, conv.shape[1], conv.shape[3]])

        bias = tf.nn.bias_add(conv_reduced, b)
        c = tf.nn.relu(bias)

        d = tf.nn.dropout(c, keep_prob=drop_rate)
        return d

    def _create_lstm_layer(self, lstm_units, conv_input):
        lstm_cell = tf.nn.rnn_cell.LSTMCell(lstm_units)
        sequence = tf.unstack(conv_input, axis=1)
        (_, (h, _)) = tf.nn.static_rnn(lstm_cell, sequence, dtype=tf.float32)

        return h

In [5]:
categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
num_classes = len(categories)

In [6]:
vocab, embd = loadGloVe(vocabPath, 100)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
# embedding = np.asarray(embd)
embedding = embd
word_to_id = dict(zip(vocab, range(vocab_size)))
len(embedding),embedding_dim,vocab_size

Loading GloVe!
Completed!


(400000, 100, 400000)

In [18]:
cat_to_id = {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

In [7]:
testData = readfile(testDataPath)
trainData = readfile(trainDataPath)

In [8]:
len(testData),len(trainData)

(500, 5452)

In [9]:
trainData[:2],testData[:2]

(array([[list(['What', 'two', 'historical', 'figures', ',', 'who', 'fought', 'each', 'other', 'in', 'a', 'famous', 'battle', ',', 'each', 'have', 'a', 'food', 'named', 'after', 'them', '?']),
         'HUM'],
        [list(['What', 'is', 'a', 'fear', 'of', 'strong', 'light', '?']),
         'ENTY']], dtype=object),
 array([[list(['Who', 'developed', 'the', 'vaccination', 'against', 'polio', '?']),
         'HUM'],
        [list(['What', 'is', 'the', 'rainiest', 'place', 'on', 'Earth', '?']),
         'LOC']], dtype=object))

In [10]:
trainData = np.r_[trainData,testData]
len(trainData)

5952

In [11]:
seq_length = 0
for content in trainData[:,0]:
    if seq_length < len(content):
        seq_length = len(content)   # seq_length = 35

In [12]:
seq_length

37

In [19]:
def process_file(contents, labels, word_to_id, cat_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad


def get_time_dif(start_time):
    """获取已使用时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))

In [14]:
learning_rate = 1e-3
dropout_keep_prob = 0.5

# 输入内容及对应的标签
input_x = tf.placeholder(tf.int32, [None, seq_length], name='input_x')
input_y = tf.placeholder(tf.float32, [None, num_classes], name='input_y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

model = ZhouCLSTMModel(embedding, drop_rate = keep_prob)
fc = model(input_x)
model.summary()

# 分类器
logits = tf.layers.dense(fc, num_classes, name='fc2')
y_pred_cls = tf.argmax(tf.nn.softmax(logits), 1)  # 预测类别 tf.argmax：返回每一行或每一列的最大值 1为里面（每一行），0为外面（每一列）

# 损失函数，交叉熵
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=input_y)
loss = tf.reduce_mean(cross_entropy)
# 优化器
optim = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# 准确率
correct_pred = tf.equal(tf.argmax(input_y, 1), y_pred_cls)
acc = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

saver = tf.train.Saver()

embedding: (?, 37, 100)
conv: (?, 35, 300)
lstm: (?, 150)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [15]:
def batch_iter(x_pad, y_pad, batch_size):
    """生成批次数据"""
    data_len = len(x_pad)
    num_batch = int((data_len - 1) / batch_size) + 1
    # np.arange()生成0到data_len的等差数列，默认等差为1；np.random.permutation()打乱生成的等差序列的顺序
    # 下面三句语句是为了将训练或测试文本的顺序打乱，因为原文本中每个分类的样本全部挨在一起，这样每个batch训练的都是同一个分类，不太好，打乱后每个batch可包含不同分类
    indices = np.random.permutation(np.arange(data_len))
    x_shuffle = x_pad[indices]
    y_shuffle = y_pad[indices]

    # 返回所有batch的数据
    for i in range(num_batch):
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, data_len)
        yield x_shuffle[start_id:end_id], y_shuffle[start_id:end_id]
        
        
def evaluate(sess, x_pad, y_pad, loss1, acc1, batch_size):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_pad)
    batch_eval = batch_iter(x_pad, y_pad, batch_size)  # 128
    total_loss = 0.0
    total_acc = 0.0
    for x_batch1, y_batch1 in batch_eval:
        batch_len = len(x_batch1)
        feed_dict1 = {input_x: x_batch1, input_y: y_batch1, keep_prob: 1.0}
        lossTmp, accTmp = sess.run([loss1, acc1], feed_dict=feed_dict1)
        total_loss += lossTmp * batch_len
        total_acc += accTmp * batch_len

    return total_loss / data_len, total_acc / data_len


def model_train(x_train, y_train, split_type, fold_id):
    
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 创建session
    session = tf.Session()
    session.run(tf.global_variables_initializer())

    print('Training and evaluating...')
    start_time = time.time()
    total_batch = 0  # 总批次
    best_acc_train = 0.0  # 最佳验证集准确率
    last_improved = 0  # 记录上一次提升批次
    num_epochs = 50
    batch_size = 128
    require_improvement = 500  # 如果超过1000轮未提升，提前结束训练
    print_per_batch = 30  # 每多少轮输出一次结果
    flag = False

    for epoch in range(num_epochs):  # 20
        print('Epoch:', epoch + 1)
        batch_train = batch_iter(x_train, y_train, batch_size)
        for x_batch, y_batch in batch_train:
            feed_dict = {input_x: x_batch, input_y: y_batch, keep_prob: dropout_keep_prob}
            session.run(optim, feed_dict=feed_dict)  # 运行优化
            total_batch += 1

            if total_batch % print_per_batch == 0:
                # 每多少轮次输出在训练集和验证集上的性能
                feed_dict[keep_prob] = 1.0
                loss_train, acc_train = session.run([loss, acc], feed_dict=feed_dict)
                # loss_val, acc_val = evaluate(session, x_dev, y_dev, loss, acc)
                if acc_train > best_acc_train:
                    # 保存最好结果
                    best_acc_train = acc_train
                    last_improved = total_batch
                    saver.save(sess=session, save_path=save_path)
                    improved_str = '*'
                else:
                    improved_str = ''

                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6}, Train Loss: {1:>6.2}, Train Acc: {2:>7.2%}, Time: {3} {4}'
                print(msg.format(total_batch, loss_train, acc_train, time_dif, improved_str))

            if total_batch - last_improved > require_improvement:
                # 验证集正确率长期不提升，提前结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break  # 跳出循环
        if flag:  # 同上
            break

    session.close()
    return best_acc_train


def model_evaluate(x_test, y_test, split_type, fold_id, categories, batch_size=64):
        
    save_path = "%s/%s/%s/%s" % (savePath, split_type, fold_id, fold_id)
    # 读取保存的模型
    session = tf.Session()
    saver.restore(sess=session, save_path=save_path)
    start_time = time.time()
    print('Testing...')
    loss_test, acc_test = evaluate(session, x_test, y_test, loss, acc, batch_size)
    msg = 'Test Loss: {0:>6.2}, Test Acc: {1:>7.2%}'
    print(msg.format(loss_test, acc_test))

    test_data_len = len(x_test)
    test_num_batch = int((test_data_len - 1) / batch_size) + 1

    y_test_cls = np.argmax(y_test, 1)  # 获得类别
    y_test_pred_cls = np.zeros(shape=len(x_test), dtype=np.int32)  # 保存预测结果  len(x_test) 表示有多少个文本

    for i in range(test_num_batch):  # 逐批次处理
        start_id = i * batch_size
        end_id = min((i + 1) * batch_size, test_data_len)
        feed_dict = {
            input_x: x_test[start_id:end_id],
            keep_prob: 1.0
        }
        y_test_pred_cls[start_id:end_id] = session.run(y_pred_cls, feed_dict=feed_dict)

    # 评估
    print("Precision, Recall and F1-Score...")
    print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
    '''
    sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
        y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
        y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
        labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
        target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
        原文链接：https://blog.csdn.net/akadiao/article/details/78788864
    '''

    # 混淆矩阵
    print("Confusion Matrix...")
    cm = metrics.confusion_matrix(y_test_cls, y_test_pred_cls)
    '''
    混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
    这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
    https://blog.csdn.net/u011734144/article/details/80277225
    '''
    print(cm)

    time_dif = get_time_dif(start_time)
    print("Time usage:", time_dif)
    session.close()

    return acc_test

In [24]:
kf = KFold(n_splits=10)
test_acc = train_10_fold(trainData, "trec")

5952 37 37
Fold:  1
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.5, Train Acc:  50.00%, Time: 0:00:14 *
Iter:     60, Train Loss:    1.5, Train Acc:  23.44%, Time: 0:00:17 
Epoch: 2
Iter:     90, Train Loss:    1.2, Train Acc:  51.56%, Time: 0:00:32 *
Iter:    120, Train Loss:    1.1, Train Acc:  53.12%, Time: 0:00:43 *
Iter:    150, Train Loss:    1.0, Train Acc:  56.25%, Time: 0:00:56 *
Epoch: 3
Iter:    180, Train Loss:    0.8, Train Acc:  75.00%, Time: 0:01:09 *
Iter:    210, Train Loss:   0.72, Train Acc:  79.69%, Time: 0:01:19 *
Iter:    240, Train Loss:   0.59, Train Acc:  78.12%, Time: 0:01:22 
Epoch: 4
Iter:    270, Train Loss:   0.59, Train Acc:  81.25%, Time: 0:01:33 *
Iter:    300, Train Loss:   0.54, Train Acc:  76.56%, Time: 0:01:36 
Iter:    330, Train Loss:    0.8, Train Acc:  68.75%, Time: 0:01:39 
Epoch: 5
Iter:    360, Train Loss:   0.39, Train Acc:  84.38%, Time: 0:01:50 *
Iter:    390, Train Loss:   0.43, Train Acc:  84.38%, Time: 0:01:54 
Ite

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/trec/1/1


Testing...
Test Loss:    0.6, Test Acc:  78.52%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.60      0.75         5
        DESC       0.73      0.73      0.73       131
        ENTY       0.76      0.79      0.78       141
         HUM       0.84      0.77      0.80       139
         LOC       0.74      0.78      0.76        85
         NUM       0.88      0.88      0.88        95

   micro avg       0.79      0.79      0.79       596
   macro avg       0.82      0.76      0.78       596
weighted avg       0.79      0.79      0.79       596

Confusion Matrix...
[[  3   2   0   0   0   0]
 [  0  96  18   6   9   2]
 [  0  12 112   9   4   4]
 [  0   9  14 107   6   3]
 [  0  10   1   5  66   3]
 [  0   3   3   1   4  84]]
Time usage: 0:00:01
Fold:  2
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.5, Train Acc:  39.06%, Time: 0:00:16 *
Iter:     60, Train Loss:    1.3, Train Acc:  42.19%, Tim

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/trec/2/2


Testing...
Test Loss:   0.97, Test Acc:  79.36%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.67      0.80        15
        DESC       0.81      0.67      0.73       130
        ENTY       0.70      0.84      0.76       130
         HUM       0.92      0.77      0.84       149
         LOC       0.75      0.84      0.79        82
         NUM       0.79      0.93      0.85        90

   micro avg       0.79      0.79      0.79       596
   macro avg       0.83      0.79      0.80       596
weighted avg       0.81      0.79      0.79       596

Confusion Matrix...
[[ 10   1   1   0   2   1]
 [  0  87  22   2  12   7]
 [  0   9 109   6   3   3]
 [  0   5  18 114   4   8]
 [  0   5   4   0  69   4]
 [  0   0   2   2   2  84]]
Time usage: 0:00:01
Fold:  3
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.4, Train Acc:  43.75%, Time: 0:00:13 *
Iter:     60, Train Loss:    1.3, Train Acc:  46.88%, Tim

...

Testing...
Test Loss:   0.93, Test Acc:  79.33%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       0.73      0.80      0.76        10
        DESC       0.66      0.89      0.76       135
        ENTY       0.83      0.69      0.75       138
         HUM       0.86      0.80      0.83       130
         LOC       0.86      0.73      0.79        92
         NUM       0.87      0.87      0.87        90

   micro avg       0.79      0.79      0.79       595
   macro avg       0.80      0.80      0.79       595
weighted avg       0.81      0.79      0.79       595

Confusion Matrix...
[[  8   2   0   0   0   0]
 [  1 120   5   4   3   2]
 [  1  28  95   8   0   6]
 [  0  10   9 104   6   1]
 [  0  14   4   4  67   3]
 [  1   7   1   1   2  78]]
Time usage: 0:00:01
Fold:  9
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.5, Train Acc:  40.62%, Time: 0:00:22 *
Iter:     60, Train Loss:    1.3, Train Acc:  43.75%, Tim

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/trec/9/9


Testing...
Test Loss:   0.76, Test Acc:  80.00%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.67      0.80         9
        DESC       0.77      0.71      0.74       119
        ENTY       0.71      0.84      0.77       153
         HUM       0.87      0.79      0.82       135
         LOC       0.86      0.78      0.82        81
         NUM       0.85      0.91      0.88        98

   micro avg       0.80      0.80      0.80       595
   macro avg       0.84      0.78      0.80       595
weighted avg       0.81      0.80      0.80       595

Confusion Matrix...
[[  6   2   1   0   0   0]
 [  0  84  26   3   2   4]
 [  0  11 128  10   0   4]
 [  0   5  16 106   5   3]
 [  0   6   5   2  63   5]
 [  0   1   4   1   3  89]]
Time usage: 0:00:01
Fold:  10
Training and evaluating...
Epoch: 1
Iter:     30, Train Loss:    1.6, Train Acc:  28.12%, Time: 0:00:20 *
Iter:     60, Train Loss:    1.4, Train Acc:  37.50%, Ti

INFO:tensorflow:Restoring parameters from s3://corpus-2/model/trec/10/10


Testing...
Test Loss:    0.6, Test Acc:  82.86%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.80      0.89        10
        DESC       0.77      0.94      0.84       154
        ENTY       0.84      0.75      0.79       121
         HUM       0.82      0.91      0.87        82
         LOC       0.78      0.73      0.75        95
         NUM       0.95      0.80      0.87       133

   micro avg       0.83      0.83      0.83       595
   macro avg       0.86      0.82      0.84       595
weighted avg       0.84      0.83      0.83       595

Confusion Matrix...
[[  8   2   0   0   0   0]
 [  0 144   7   1   1   1]
 [  0  10  91   9   7   4]
 [  0   3   2  75   2   0]
 [  0  18   4   3  69   1]
 [  0  11   4   3   9 106]]
Time usage: 0:00:01
[0.7852348997288903, 0.7936241606737944, 0.7697479000612467, 0.7966386551616572, 0.8067226897768613, 0.7563025207078757, 0.8285714294730114, 0.7932773111247214, 0.800000000